![image info](data/MA.avif)

**Retrieval augmented generation (RAG)** with [Gemini File Search](https://blog.google/technology/developers/file-search-gemini-api/) tool 

The PDF is a copy of an 1887 book with archaic fonts an aged pages.
The latest RAG tool from Gemini is able to transcibe and provide insights on this material in one cell of Python Code

In [ ]:
import time
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)
store = client.file_search_stores.create()

upload_op = client.file_search_stores.upload_to_file_search_store(
    file_search_store_name=store.name,
    file='./data/meditationsofmar00marc.pdf'
)

while not upload_op.done:
  time.sleep(5)
  upload_op = client.operations.get(upload_op)

# Use the file search store as a tool in your generation call
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents='who was marcus aurelius?',
    config=types.GenerateContentConfig(
        tools=[types.Tool(
            file_search=types.FileSearch(
                file_search_store_names=[store.name]
            )
        )]
    )
)
print(response.text)

# Support your response with links to the grounding sources.
grounding = response.candidates[0].grounding_metadata
if not grounding:
 print('No grounding sources found')
else:
 sources = {c.retrieved_context.title for c in grounding.grounding_chunks}
 print('Sources:', *sources)

Marcus Aurelius Antoninus was a Roman Emperor who reigned from 161 A.D. until his death in 180 A.D.. He was the adopted son of Emperor Antoninus Pius and received a thorough education, excelling in poetry and rhetoric, though his preferred area of study was philosophy. He is often referred to as the "philosopher king," as described by Plato, embodying a ruler who loved philosophy more than power and the solitude of study more than imperial grandeur.

His reign was marked by considerable challenges, including wars and troubles that continued until his final days. Despite the demanding nature of his rule, he consistently applied his philosophical principles to his actions. Marcus Aurelius was a follower of Stoicism, a philosophy that greatly influenced his conduct and thinking. His personal reflections, known as "Meditations," offer insight into his inner life and philosophical views. In these writings, he emphasizes the importance of acknowledging obligations to parents, friends, and te